In [261]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Starting from raw dataset in need of cleaning

## Preprocessing the data

In [262]:
# Select the columns we want to explore
full = pd.read_csv("DSI_kickstarterscrape_dataset.txt")
relevant = full[['project id', 'name', 'category', 'location', 'status', 'goal', 'pledged', 'funded percentage', 'backers', 'funded date', 'levels', 'updates', 'comments', 'duration']]

print(relevant.describe())
relevant.head()

project id          goal       pledged  funded percentage  \
count  4.595700e+04  4.595700e+04  4.594500e+04       45957.000000   
mean   1.080800e+09  1.194271e+04  4.980750e+03           1.850129   
std    6.218057e+08  1.887583e+05  5.674162e+04          88.492706   
min    3.940900e+04  1.000000e-02  0.000000e+00           0.000000   
25%    5.438962e+08  1.800000e+03  1.960000e+02           0.044000   
50%    1.078345e+09  4.000000e+03  1.310000e+03           1.000000   
75%    1.621596e+09  9.862000e+03  4.165000e+03           1.115640   
max    2.147460e+09  2.147484e+07  1.026684e+07       15066.000000   

            backers        levels      updates      comments      duration  
count  45957.000000  45957.000000  45957.00000  45957.000000  45957.000000  
mean      69.973192      8.004939      4.08508      8.379529     39.995547  
std      688.628479      4.233907      6.43922    174.015737     17.414458  
min        0.000000      0.000000      0.00000      0.000000      1.00

,project id,name,category,location,status,goal,pledged,funded percentage,backers,funded date,levels,updates,comments,duration
0,39409,WHILE THE TREES SLEEP,Film & Video,"Columbia, MO",successful,10500.0,11545.0,1.099524,66,"Fri, 19 Aug 2011 19:28:17 -0000",7,10,2,30.00
1,126581,Educational Online Trading Card Game,Games,"Maplewood, NJ",failed,4000.0,20.0,0.005000,2,"Mon, 02 Aug 2010 03:59:00 -0000",5,6,0,47.18
2,138119,STRUM,Film & Video,"Los Angeles, CA",live,20000.0,56.0,0.002800,3,"Fri, 08 Jun 2012 00:00:31 -0000",10,1,0,28.00
3,237090,GETTING OVER - One son's search to finally kno...,Film & Video,"Los Angeles, CA",successful,6000.0,6535.0,1.089167,100,"Sun, 08 Apr 2012 02:14:00 -0000",13,4,0,32.22
4,246101,The Launch of FlyeGrlRoyalty &quot;The New Nam...,Fashion,"Novi, MI",failed,3500.0,0.0,0.000000,0,"Wed, 01 Jun 2011 15:25:39 -0000",6,2,0,30.00


In [263]:
date_funded = relevant['funded date']
date_funded.head()

0    Fri, 19 Aug 2011 19:28:17 -0000
1    Mon, 02 Aug 2010 03:59:00 -0000
2    Fri, 08 Jun 2012 00:00:31 -0000
3    Sun, 08 Apr 2012 02:14:00 -0000
4    Wed, 01 Jun 2011 15:25:39 -0000
Name: funded date, dtype: object

In [264]:
# Format 'funded date' column as a datetime object
relevant['funded date'] = pd.to_datetime(date_funded, infer_datetime_format=True)

print(relevant['funded date'].describe())
relevant['funded date'].head()

count                   45957
unique                  41068
top       2012-01-01 04:59:00
freq                       44
first     2009-05-03 06:59:59
last      2012-08-01 19:36:12
Name: funded date, dtype: object


0   2011-08-19 19:28:17
1   2010-08-02 03:59:00
2   2012-06-08 00:00:31
3   2012-04-08 02:14:00
4   2011-06-01 15:25:39
Name: funded date, dtype: datetime64[ns]

In [265]:
# Correct misspelled category 
relevant['category'] = relevant['category'].replace('Film &amp; Video','Film & Video')

print(relevant['category'].unique())
relevant['category'].describe()

['Film & Video' 'Games' 'Fashion' 'Music' 'Art' 'Technology' 'Dance'
 'Publishing' 'Theater' 'Comics' 'Design' 'Photography' 'Food']


count            45957
unique              13
top       Film & Video
freq             13551
Name: category, dtype: object

In [266]:
# We are only looking at successful, failed, and canceled campaigns
print(relevant['status'].unique())
status_mapping = {'successful':1, 'failed':0, 'canceled':0}

relevant['status'] = relevant['status'].map(status_mapping)
relevant['status'].unique()

['successful' 'failed' 'live' 'canceled' 'suspended']


array([ 1.,  0., nan])

In [267]:
# Filter for data of successful, failed, or canceled campaigns
mask = relevant['status'].notna()
relevant = relevant[mask]

print(relevant.describe())
relevant.head()

project id        status          goal       pledged  \
count  4.202400e+04  42024.000000  4.202400e+04  4.202400e+04   
mean   1.081359e+09      0.546569  1.044899e+04  5.119146e+03   
std    6.216207e+08      0.497833  1.477671e+05  5.903410e+04   
min    3.940900e+04      0.000000  1.000000e-02  0.000000e+00   
25%    5.441116e+08      0.000000  1.677250e+03  2.200000e+02   
50%    1.080229e+09      1.000000  4.000000e+03  1.445000e+03   
75%    1.621810e+09      1.000000  8.888000e+03  4.396250e+03   
max    2.147460e+09      1.000000  2.147484e+07  1.026684e+07   

       funded percentage       backers        levels       updates  \
count       42024.000000  42024.000000  42024.000000  42024.000000   
mean            1.980679     70.986246      7.952408      4.334285   
std            92.536438    712.465184      4.190945      6.626322   
min             0.000000      0.000000      0.000000      0.000000   
25%             0.051218      5.000000      5.000000      0.000000   
50%

,project id,name,category,location,status,goal,pledged,funded percentage,backers,funded date,levels,updates,comments,duration
0,39409,WHILE THE TREES SLEEP,Film & Video,"Columbia, MO",1.0,10500.0,11545.0,1.099524,66,2011-08-19 19:28:17,7,10,2,30.00
1,126581,Educational Online Trading Card Game,Games,"Maplewood, NJ",0.0,4000.0,20.0,0.005000,2,2010-08-02 03:59:00,5,6,0,47.18
3,237090,GETTING OVER - One son's search to finally kno...,Film & Video,"Los Angeles, CA",1.0,6000.0,6535.0,1.089167,100,2012-04-08 02:14:00,13,4,0,32.22
4,246101,The Launch of FlyeGrlRoyalty &quot;The New Nam...,Fashion,"Novi, MI",0.0,3500.0,0.0,0.000000,0,2011-06-01 15:25:39,6,2,0,30.00
5,316217,Dinner Party - a short film about friendship.....,Film & Video,"Portland, OR",1.0,3500.0,3582.0,1.023331,39,2011-06-22 13:33:00,7,8,0,21.43


In [268]:
# Remove rows containing 'pledged' outliers
print(relevant['pledged'].describe())
stdev = relevant['pledged'].std()
avg = relevant['pledged'].mean()
outlier_threshold = avg + 3 * stdev

mask = relevant['pledged'] <= outlier_threshold
relevant = relevant[mask]
relevant['pledged'].describe()

count    4.202400e+04
mean     5.119146e+03
std      5.903410e+04
min      0.000000e+00
25%      2.200000e+02
50%      1.445000e+03
75%      4.396250e+03
max      1.026684e+07
Name: pledged, dtype: float64


count     41968.000000
mean       4152.236680
std        9458.697489
min           0.000000
25%         220.000000
50%        1441.000000
75%        4367.000000
max      179555.000000
Name: pledged, dtype: float64

In [273]:
# Create dataset of succesful campaigns
mask = relevant['status'] > 0.1
relevant_successful = relevant[mask]

print(relevant_successful.describe())
relevant_successful.head()

project id   status           goal        pledged  funded percentage  \
count  2.291300e+04  22913.0   22913.000000   22913.000000       22913.000000   
mean   1.081482e+09      1.0    5213.319729    6856.197530           3.506405   
std    6.207200e+08      0.0    7750.177442   11873.413549         125.293682   
min    3.940900e+04      1.0       0.010000       1.000000           0.999830   
25%    5.459046e+08      1.0    1250.000000    1652.000000           1.033667   
50%    1.087977e+09      1.0    3000.000000    3487.000000           1.109200   
75%    1.619303e+09      1.0    6000.000000    7199.000000           1.317308   
max    2.147460e+09      1.0  150000.000000  179555.000000       15066.000000   

            backers        levels       updates      comments      duration  
count  22913.000000  22913.000000  22913.000000  22913.000000  22913.000000  
mean      95.873871      8.482477      6.652686      8.247763     37.951390  
std      199.791823      4.422888      7.6604

,project id,name,category,location,status,goal,pledged,funded percentage,backers,funded date,levels,updates,comments,duration
0,39409,WHILE THE TREES SLEEP,Film & Video,"Columbia, MO",1.0,10500.0,11545.0,1.099524,66,2011-08-19 19:28:17,7,10,2,30.00
3,237090,GETTING OVER - One son's search to finally kno...,Film & Video,"Los Angeles, CA",1.0,6000.0,6535.0,1.089167,100,2012-04-08 02:14:00,13,4,0,32.22
5,316217,Dinner Party - a short film about friendship.....,Film & Video,"Portland, OR",1.0,3500.0,3582.0,1.023331,39,2011-06-22 13:33:00,7,8,0,21.43
7,407836,Help APORTA continue to make handwoven/knit ac...,Fashion,"Chicago, IL",1.0,2000.0,2180.0,1.090000,46,2011-12-30 04:36:53,7,13,5,30.00
8,436325,Music - Comedy - Album!,Music,"Chicago, IL",1.0,1000.0,1125.0,1.125000,30,2010-04-18 04:59:00,12,10,1,67.53


# Look for insights
# Notes on the dataset:
• Composed of completed campaigns only.  
• All figures comparing 'average pledged' use data exclusively from successful campaigns.  
• Outlier rows with 'pledged' value greater than 3 x standard deviation have been removed.  
• The data feeding each figure is filtered for quality. Because the plots are of aggregate functions, each point along the x axis is really composed of many data points from the original dataset. All points containing information from less than the mean number of points used per aggregate are discarded, generally. 

• Data from May 2009 to July 2012  
• The dataset used to evaluate success rate has 42,000 campaigns documented.  
• The dataset used to evaluate average pledged has 22,900 campaigns documented.

# By Category

# Key Takeaways:
## High risk, high reward
### • Design, Games
## Safest Categories
### • Theater, Art

## Average Pledged

In [275]:
n_counts = relevant_successful.groupby(['category'])['pledged'].count()
mean_data = relevant_successful.groupby(['category'])['pledged'].mean()

categories = pd.concat([n_counts,mean_data], keys=['counts', 'avg_pledged'], names=['counts', 'avg_pledged'], axis=1)
categories

,counts,avg_pledged
category,,
Art,2106,4482.974359
Comics,519,6829.635838
Dance,527,3419.222011
Design,708,17116.413842
Fashion,334,6065.311377
Film & Video,6416,8312.976777
Food,654,8915.825688
Games,620,13937.348387
Music,6790,5147.648012


In [276]:
print(f"Mean counts per category: {categories['counts'].mean()}")
counts_mean = categories['counts'].mean()
threshold = counts_mean * 0.25
mask = (categories['counts'] >  threshold)
categories_high_counts = categories[mask]

Mean counts per category: 1762.5384615384614


In [277]:
categories_high_counts = categories_high_counts.sort_values('avg_pledged', ascending=False)
fig = px.bar(categories_high_counts,categories_high_counts.index,'avg_pledged')
fig.update_layout(hovermode="x unified", title="Average Pledged by Category")
fig.show()

## Success Rate

In [253]:
n_counts = relevant.groupby(['category'])['status'].count()
mean_data = relevant.groupby(['category'])['status'].sum()
success_ratio = mean_data/n_counts

categories = pd.concat([n_counts,success_ratio], keys=['campaigns', 'success rate'], names=['campaigns', 'success rate'], axis=1)
categories = categories.sort_values('success rate', ascending=False)

In [255]:
print(f"Mean counts per category: {categories['campaigns'].mean()}")
counts_mean = categories['campaigns'].mean()
threshold = counts_mean * 0.25
mask = (categories['campaigns'] > threshold)
categories_high_counts = categories[mask]

Mean counts per category: 3232.6153846153848


In [256]:
fig = px.bar(categories_high_counts,categories_high_counts.index,'success rate')
fig.update_layout(hovermode="x unified", title="Success Rate by Category")
fig.show()

# By Fundraiser Duration

# Key Takeaways:
## Shorter campaigns are more likely to succeed.
## Keep campaigns under 30 days to stay above 60% success rate.
### • Pearson Correlation Coefficient: -.875

## Average Pledged

In [278]:
n_counts = relevant_successful.groupby(['duration'])['pledged'].count()
mean_data = relevant_successful.groupby(['duration'])['pledged'].mean()

duration = pd.concat([n_counts,mean_data], keys=['counts', 'avg_pledged'], names=['counts', 'avg_pledged'], axis=1)
duration.head()

,counts,avg_pledged
duration,,
1.00,4,1547.75
1.01,2,394.00
1.03,1,70.00
1.93,1,101.00
1.94,1,18.00


In [279]:
print(f"Mean counts per duration: {duration['counts'].mean()}")
duration_mean = duration['counts'].mean()
threshold = duration_mean * 10
mask = (duration['counts'] > threshold)
duration_high_counts = duration[mask]

Mean counts per duration: 4.836006753904601


In [280]:
fig = px.scatter(duration_high_counts,duration_high_counts.index,'avg_pledged')
fig.update_layout(hovermode="x unified", title="Average Pledged vs Campaign Duration")
fig.show()

## Success Rate

In [281]:
n_counts = relevant.groupby(['duration'])['status'].count()
mean_data = relevant.groupby(['duration'])['status'].sum()
success_ratio = mean_data/n_counts

duration = pd.concat([n_counts,success_ratio], keys=['campaigns','success rate'], names=['campaigns','success rate'], axis=1)
duration.head()

,campaigns,success rate
duration,,
1.00,5,0.8
1.01,2,1.0
1.03,1,1.0
1.49,1,0.0
1.93,2,0.5


In [282]:
print(f"Mean counts per duration: {duration['campaigns'].mean()}")
counts_mean = duration['campaigns'].mean()
threshold = counts_mean * 10
mask = (duration['campaigns'] > threshold)
duration_high_counts = duration[mask]

Mean counts per duration: 7.134988099285957


In [283]:
fig = px.scatter(duration_high_counts,duration_high_counts.index,'success rate')
fig.update_layout(hovermode="x unified", title="Success Rate by Campaign Length")
fig.show()

In [284]:
duration_corr_df = duration_high_counts[['success rate']].reset_index()
duration_corr_df.corr()

,duration,success rate
duration,1.000000,-0.874985
success rate,-0.874985,1.000000


# By Goal

# Key Takeaways:
## A high goal allows for greater earning potential
### • Pearson Correlation Coefficient: .997
## Find a balance between your projects funding needs and organizing a campaign that is likely to succeed
### • Exponential drop off of success rate / goal set

## Average Pledged

In [285]:
n_counts = relevant_successful.groupby(['goal'])['pledged'].count()
mean_data = relevant_successful.groupby(['goal'])['pledged'].mean()

goal = pd.concat([n_counts,mean_data], keys=['counts','avg_pledged'], names=['counts','avg_pledged'], axis=1)
goal.head()

,counts,avg_pledged
goal,,
0.01,1,100.000000
1.00,26,1292.923077
3.00,2,48.500000
5.00,7,133.285714
10.00,9,214.888889


In [286]:
relevant[relevant['pledged']==relevant['pledged'].max()]

,project id,name,category,location,status,goal,pledged,funded percentage,backers,funded date,levels,updates,comments,duration
43316,2031031861,DAVID LYNCH DOCUMENTARY,Film & Video,"Hollywood, CA",1.0,30000.0,179555.0,5.985166,1397,2012-04-11 20:43:14,11,5,91,44.96


In [287]:
print(f"Mean counts per goal: {goal['counts'].mean()}")
counts_mean = goal['counts'].mean()
threshold = counts_mean
mask = (goal['counts'] > threshold)
goal_high_counts = goal[mask]

Mean counts per goal: 23.50051282051282


In [288]:
goal_high_counts[goal_high_counts['counts']==goal_high_counts['counts'].max()]

,counts,avg_pledged
goal,,
5000.0,1981,6355.602221


In [289]:
fig = px.scatter(goal_high_counts,goal_high_counts.index,'avg_pledged')
fig.update_layout(hovermode="x unified", title="Project Goal vs Average Pledged")
fig.show()

In [290]:
goal_corr_df = goal_high_counts[['avg_pledged']].reset_index()
goal_corr_df.corr()

,goal,avg_pledged
goal,1.000000,0.997191
avg_pledged,0.997191,1.000000


## Success Rate

In [291]:
n_counts = relevant.groupby(['goal'])['status'].count()
mean_data = relevant.groupby(['goal'])['status'].sum()
success_ratio = mean_data/n_counts

goal = pd.concat([n_counts,success_ratio], keys=['campaigns','success rate'], names=['campaigns','success rate'], axis=1)
goal.head()

,campaigns,success rate
goal,,
0.01,1,1.000000
0.50,1,0.000000
1.00,28,0.928571
1.85,1,0.000000
2.00,1,0.000000


In [292]:
print(f"Mean counts per goal: {goal['campaigns'].mean()}")
counts_mean = goal['campaigns'].mean()
threshold = counts_mean
mask = (goal['campaigns'] > threshold)
goal = goal[mask]

Mean counts per goal: 24.557050906963138


In [293]:
fig = px.scatter(goal,goal.index,'success rate')
fig.update_layout(hovermode="x unified", title="Success Rate by Campaign Goal")
fig.show()

# By Timing

# Key Takeaways:
## Any time is a good time to launch a campaign
### • Success rate is equal accross all months.
### • Face more competition earlier in the year, but Average Pledged per month shows no clear trend.

In [294]:
# Derive new columns to investigate month, day and yearly influence
month = pd.Series([date.month for date in relevant['funded date']])
day = pd.Series([date.day for date in relevant['funded date']])
year = pd.Series([date.year for date in relevant['funded date']])
date_add_ons = pd.concat([month,day,year],axis=1,keys=['month','day','year'])

In [295]:
relevant_dates = pd.concat([relevant,date_add_ons], axis=1)
relevant_dates_successful = pd.concat([relevant_successful,date_add_ons], axis=1)

## Month

In [296]:
n_counts = relevant_dates_successful.groupby(['month'])['pledged'].count()
mean_data = relevant_dates_successful.groupby(['month'])['pledged'].mean()

month = pd.concat([n_counts,mean_data], keys=['counts','avg_pledged'], names=['counts','avg_pledged'], axis=1)
month

,counts,avg_pledged
month,,
1.0,1536,7057.056641
2.0,1652,6858.435230
3.0,2256,6540.094858
4.0,2617,6730.123424
5.0,2903,7368.756459
6.0,1447,6636.933656
7.0,1405,6804.054804
8.0,1499,7071.765844
9.0,1323,6504.583522


In [297]:
fig = px.bar(month,x=month.index,y='avg_pledged',color='counts')
fig.update_layout(title="Average Pledged per Month")
fig.show()

In [298]:
n_counts = relevant_dates.groupby(['month'])['status'].count()
mean_data = relevant_dates.groupby(['month'])['status'].sum()
success_ratio = mean_data/n_counts

month = pd.concat([n_counts,success_ratio], keys=['campaigns','success rate'], names=['campaigns','success rate'], axis=1)
month.head()

,campaigns,success rate
month,,
1.0,2816,0.545455
2.0,2952,0.559621
3.0,4153,0.543222
4.0,4656,0.562070
5.0,5220,0.556130


In [299]:
fig = px.bar(month,x=month.index,y='success rate',color='campaigns')
fig.update_layout(title="Success Rate per Month")
fig.show()

## Day

In [300]:
n_counts = relevant_dates_successful.groupby(['day'])['pledged'].count()
mean_data = relevant_dates_successful.groupby(['day'])['pledged'].mean()

day = pd.concat([n_counts,mean_data], keys=['counts','avg_pledged'], names=['counts','avg_pledged'], axis=1)
day.head()

,counts,avg_pledged
day,,
1.0,1475,6747.975593
2.0,977,7217.863869
3.0,642,6794.624611
4.0,651,7484.533026
5.0,651,6591.711214


In [301]:
fig = px.bar(day,x=day.index,y='avg_pledged')
fig.update_layout(title="Average Pledged per Day of the Month")
fig.show()

# By Location

# Key Takeaways:
## US is the most active country
### • Not much insight here

## Average Pledged

In [302]:
# Location column is corrupted. We will refine our aggregation further.
n_counts = relevant_dates_successful.groupby(['location'])['pledged'].count()
mean_data = relevant_dates_successful.groupby(['location'])['pledged'].mean()

location = pd.concat([n_counts,mean_data], keys=['counts','avg_pledged'], names=['counts','avg_pledged'], axis=1)
location.head()

,counts,avg_pledged
location,,
"Aberdeen, WA",1,6550.0
"Abilene, TX",1,695.0
"Abuja, Nigeria",1,2389.0
"Accra, Ghana",6,4787.5
"Adams, MA",1,3600.0


In [303]:
# Take only the state or country listed
print(location.index.unique())
location_column_cleaned = pd.Series(location.index)
location_column_cleaned = location_column_cleaned.apply(lambda s: s.split(',')[-1])
location_column_cleaned.head()

Index(['Aberdeen, WA', 'Abilene, TX', 'Abuja, Nigeria', 'Accra, Ghana',
       'Adams, MA', 'Addis Ababa, Ethiopia', 'Addison, IL', 'Addison, TX',
       'Adelphi, MD', 'Adirondack, NY',
       ...
       'Yosemite National Park, CA', 'Youngstown, OH', 'Ypsilanti, MI',
       'Yuba City, CA', 'Yucaipa, CA', 'Yucat��΍n, Mexico', 'Yucca Valley, CA',
       'Zapotitl��΍n Palmas, Mexico', 'Zurich, Switzerland', '�ۊ�_re, Sweden'],
      dtype='object', name='location', length=2662)


0          WA
1          TX
2     Nigeria
3       Ghana
4          MA
Name: location, dtype: object

In [304]:
location.index = location_column_cleaned
location.head()

,counts,avg_pledged
location,,
WA,1,6550.0
TX,1,695.0
Nigeria,1,2389.0
Ghana,6,4787.5
MA,1,3600.0


In [305]:
# Aggregate again with properly formatted location column
n_counts = location.groupby(['location'])['avg_pledged'].count()
mean_data = location.groupby(['location'])['avg_pledged'].mean()

location = pd.concat([n_counts,mean_data], keys=['counts','avg_pledged'], names=['counts','avg_pledged'], axis=1)
location.head()

,counts,avg_pledged
location,,
AK,13,7076.615385
AL,20,6512.535088
AR,16,4213.459754
AZ,26,7466.307476
Afghanistan,3,5531.476190


In [306]:
mask = (location['counts'] > 20)
location_high_counts = location[mask]

In [307]:
fig = px.pie(location_high_counts,names=location_high_counts.index,values='avg_pledged')
fig.update_layout(title="Average Pledged per Location")
fig.update_traces(textposition='inside', textinfo='label')
fig.show()

## Success Rate

In [308]:
# Location column is corrupted. We will refine our aggregation further.
n_counts = relevant_dates.groupby(['location'])['status'].count()
mean_data = relevant_dates.groupby(['location'])['status'].mean()

location = pd.concat([n_counts,mean_data], keys=['campaigns','success rate'], names=['campaigns','success rate'], axis=1)
location.head()

,campaigns,success rate
location,,
"1, LA",2,0.000000
"10, Middleburg, MD",1,0.000000
"Aberdeen, SD",2,0.000000
"Aberdeen, WA",1,1.000000
"Abilene, TX",3,0.333333


In [309]:
# Take only the state or country listed
print(location.index.unique())
location_column_cleaned = pd.Series(location.index)
location_column_cleaned = location_column_cleaned.apply(lambda s: s.split(',')[-1])
location_column_cleaned.head()

Index(['1, LA', '10, Middleburg, MD', 'Aberdeen, SD', 'Aberdeen, WA',
       'Abilene, TX', 'Absecon, NJ', 'Abu Dhabi, United Arab Emirates',
       'Abuja, Nigeria', 'Accra, Ghana', 'Ackerly, TX',
       ...
       'Zapotitl��΍n Palmas, Mexico', 'Zephyrhills, FL', 'Zirconia, NC',
       'Zurich, Switzerland', 'Z���_rich District, Switzerland',
       '�Ȋ�_��ε��α ���_�Ȋ�_��Ό�����ȁ�������Ω, Egypt', '�ۊ�_lvik, Norway',
       '�ۊ�_re, Sweden', '�ۊ��stersund, Sweden', '���_�Ȋ�_���_�Ȋ��, Jordan'],
      dtype='object', name='location', length=4526)


0     LA
1     MD
2     SD
3     WA
4     TX
Name: location, dtype: object

In [310]:
location.index = location_column_cleaned
location.head()

,campaigns,success rate
location,,
LA,2,0.000000
MD,1,0.000000
SD,2,0.000000
WA,1,1.000000
TX,3,0.333333


In [311]:
# Aggregate again with properly formatted location column
n_counts = location.groupby(['location'])['success rate'].count()
mean_data = location.groupby(['location'])['success rate'].mean()

location = pd.concat([n_counts,mean_data], keys=['campaigns','success rate'], names=['campaigns','success rate'], axis=1)
location.head()

,campaigns,success rate
location,,
AK,26,0.439744
AL,35,0.412330
AR,30,0.351556
AZ,56,0.302743
Afghanistan,4,0.659091


In [312]:
mask = (location['campaigns'] > 50)
location_high_counts = location[mask]

In [313]:
fig = px.pie(location_high_counts,names=location_high_counts.index,values='success rate')
fig.update_layout(title="Success Rate per Location")
fig.update_traces(textposition='inside', textinfo='label')
fig.show()

# By Number of Backers

# Key Takeaways:
## More backers, more money
### • Pearson Correlation Coefficient: .974
## 20 backers yields 62% success rate
## 40 backers yields 87% success rate

## Avg Pledged

In [314]:
n_counts = relevant_dates_successful.groupby(['backers'])['pledged'].count()
mean_data = relevant_dates_successful.groupby(['backers'])['pledged'].mean()

backers = pd.concat([n_counts,mean_data], keys=['counts','avg_pledged'], names=['counts','avg_pledged'], axis=1)
backers.head()

,counts,avg_pledged
backers,,
1.0,23,591.347826
2.0,24,436.208333
3.0,35,287.142857
4.0,49,553.061224
5.0,74,635.635135


In [315]:
print(f"Mean counts per backers: {backers['counts'].mean()}")
counts_mean = backers['counts'].mean()
threshold = counts_mean
mask = (backers['counts'] > threshold)
backers_high_counts = backers[mask]

Mean counts per backers: 26.397465437788018


In [316]:
fig = px.scatter(backers_high_counts,x=backers_high_counts.index,y='avg_pledged')
fig.update_layout(title="Average Pledged vs Number of Backers")
fig.show()

In [317]:
backers_corr_df = backers_high_counts[['avg_pledged']].reset_index()
backers_corr_df.corr()

,backers,avg_pledged
backers,1.000000,0.974417
avg_pledged,0.974417,1.000000


## Success Rate

In [318]:
n_counts = relevant_dates.groupby(['backers'])['status'].count()
mean_data = relevant_dates.groupby(['backers'])['status'].sum()
success_ratio = mean_data/n_counts

backers = pd.concat([n_counts,success_ratio], keys=['campaigns','success rate'], names=['campaigns','success rate'], axis=1)
backers.head()

,campaigns,success rate
backers,,
0.0,3481,0.000000
1.0,2227,0.010328
2.0,1678,0.014303
3.0,1273,0.027494
4.0,1006,0.048708


In [319]:
print(f"Mean counts per backers: {backers['campaigns'].mean()}")
counts_mean = backers['campaigns'].mean()
threshold = counts_mean
mask = (backers['campaigns'] > threshold)
backers_high_counts = backers[mask]

Mean counts per backers: 48.12844036697248


In [320]:
fig = px.scatter(backers_high_counts,x=backers_high_counts.index,y='success rate')
fig.update_layout(title="Success Rate vs Number of Backers")
fig.show()